## Loading and Importing VGG model for transfer learning

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense
from tensorflow import keras
from tensorflow.keras import layers
from keras import Sequential

## Preprocessing and loading the data

In [ ]:
train_batch = tf.keras.utils.image_dataset_from_directory(
    "", labels='inferred', label_mode='binary', image_size=(299, 299), color_mode='rgb', interpolation='lanczos3'
)
val_batch = tf.keras.utils.image_dataset_from_directory(
    "", labels='inferred', label_mode='binary', image_size=(299, 299), color_mode='rgb', interpolation='lanczos3'
 )

## Loading Xception and recreating it

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
sleeping_net = Sequential()

pretrained_model = tf.keras.applications.Xception(
    include_top=False,
    input_shape=(299, 299, 3),
    pooling='avg',
    classes=2,
    weights='imagenet',

)

for layer in pretrained_model.layers:
  layer.trainable=False

sleeping_net.add(pretrained_model)
sleeping_net.add(Flatten())
sleeping_net.add(Dense(512, activation='relu'))
sleeping_net.add(Dense(1, activation='sigmoid'))


Model Summary

In [ ]:
sleeping_net.summary()

### Compiling the model

In [ ]:
model_compile = sleeping_net.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy'],
)

### Callback functions

In [ ]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2),
    tf.keras.callbacks.ModelCheckpoint(filepath='SleepingNet.h5', save_best_only=True),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8),
]

## Fitting the model

In [ ]:
history = sleeping_net.fit(
    train_batch,
    batch_size=64,
    epochs=20,
    verbose=2,
    callbacks=callbacks,
    validation_data=val_batch,
)